# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Model-with-gpu" data-toc-modified-id="Model-with-gpu-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Model with gpu</a></div><div class="lev1 toc-item"><a href="#Model-Without-gpu" data-toc-modified-id="Model-Without-gpu-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Model Without gpu</a></div>

## Model with gpu

In [8]:
import chainer
from chainer import Chain
from chainer import optimizers
import chainer.links as L
import chainer.functions as F
from chainer import datasets, iterators, training
from chainer.training import extensions
import numpy as np

class MLP(Chain):
       def __init__(self, n_units, n_out):
           super(MLP, self).__init__()
           with self.init_scope():
               self.l1 = L.Linear(None, n_units)
               self.l2 = L.Linear(None, n_units)
               self.l3 = L.Linear(None, n_out)

       def __call__(self, x):
           h1 = F.relu(self.l1(x))
           h2 = F.relu(self.l2(h1))
           y = self.l3(h2)
           return y

model = L.Classifier(MLP(1000, 10)).to_gpu(0)  # to_gpu returns itself
optimizer = optimizers.SGD()
optimizer.setup(model)

In [15]:
data = np.random.rand(70000, 784).astype(np.float32)
target = np.random.randint(10, size=70000).astype(np.int32)
train = datasets.TupleDataset(data[:60000], target[:60000])
test = datasets.TupleDataset(data[60000:], target[60000:])
train_iter = iterators.SerialIterator(train, batch_size=100)
test_iter = iterators.SerialIterator(test, batch_size=100, repeat=False, shuffle=False)

In [ ]:
# model.to_gpu()
# batchsize = 100
# datasize = len(x_train)
# for epoch in range(20):
#     print('epoch %d' % epoch)
#     indexes = np.random.permutation(datasize)
#     for i in range(0, datasize, batchsize):
#         x = Variable(cuda.to_gpu(x_train[indexes[i : i + batchsize]]))
# #         t = Variable(cuda.to_gpu(y_train[indexes[i : i + batchsize]]))
# #         optimizer.update(model, x, t)




In [18]:
x_train

In [3]:
updater = training.StandardUpdater(train_iter, optimizer, device=0)
trainer = training.Trainer(updater, (30, 'epoch'), out='result')
trainer.extend(extensions.LogReport())
trainer.extend(extensions.snapshot(filename='snapshot_epoch-{.updater.epoch}'))
trainer.extend(extensions.snapshot_object(model.predictor, filename='model_epoch-{.updater.epoch}'))
trainer.extend(extensions.Evaluator(test_iter, model, device=0))
trainer.extend(extensions.PrintReport(['epoch', 'main/loss', 'main/accuracy', 'validation/main/loss', 'validation/main/accuracy', 'elapsed_time']))
trainer.extend(extensions.PlotReport(['main/loss', 'validation/main/loss'], x_key='epoch', file_name='loss.png'))
trainer.extend(extensions.PlotReport(['main/accuracy', 'validation/main/accuracy'], x_key='epoch', file_name='accuracy.png'))
trainer.extend(extensions.dump_graph('main/loss'))

In [4]:
trainer.run()

epoch       main/loss   main/accuracy  validation/main/loss  validation/main/accuracy  elapsed_time
1           2.31265     0.09955        2.31211               0.1006                    3.31361       
2           2.30468     0.105817       2.30965               0.1016                    6.61322       
3           2.30014     0.112767       2.30857               0.0993                    10.1756       
4           2.29644     0.119267       2.30781               0.1009                    13.6854       
5           2.29302     0.1241         2.31043               0.0999                    16.989        
6           2.29003     0.129417       2.31072               0.1021                    20.2314       
7           2.28713     0.132033       2.31092               0.1021                    23.5109       
8           2.28413     0.137883       2.31151               0.1016                    26.7419       
9           2.28139     0.1412         2.31065               0.1003                 

## Model Without gpu

In [5]:
import chainer
from chainer import Chain
from chainer import optimizers
import chainer.links as L
import chainer.functions as F
from chainer import datasets, iterators, training
from chainer.training import extensions
import numpy as np

class MLP(Chain):
       def __init__(self, n_units, n_out):
           super(MLP, self).__init__()
           with self.init_scope():
               self.l1 = L.Linear(None, n_units)
               self.l2 = L.Linear(None, n_units)
               self.l3 = L.Linear(None, n_out)

       def __call__(self, x):
           h1 = F.relu(self.l1(x))
           h2 = F.relu(self.l2(h1))
           y = self.l3(h2)
           return y

model = L.Classifier(MLP(1000, 10))  # to_gpu returns itself
optimizer = optimizers.SGD()
optimizer.setup(model)

In [6]:
data = np.random.rand(70000, 784).astype(np.float32)
target = np.random.randint(10, size=70000).astype(np.int32)
train = datasets.TupleDataset(data[:60000], target[:60000])
test = datasets.TupleDataset(data[60000:], target[60000:])
train_iter = iterators.SerialIterator(train, batch_size=100)
test_iter = iterators.SerialIterator(test, batch_size=100, repeat=False, shuffle=False)

In [7]:
updater = training.StandardUpdater(train_iter, optimizer)
trainer = training.Trainer(updater, (2, 'epoch'), out='result')
trainer.extend(extensions.LogReport())
trainer.extend(extensions.snapshot(filename='snapshot_epoch-{.updater.epoch}'))
trainer.extend(extensions.snapshot_object(model.predictor, filename='model_epoch-{.updater.epoch}'))
trainer.extend(extensions.Evaluator(test_iter, model))
trainer.extend(extensions.PrintReport(['epoch', 'main/loss', 'main/accuracy', 'validation/main/loss', 'validation/main/accuracy', 'elapsed_time']))
trainer.extend(extensions.PlotReport(['main/loss', 'validation/main/loss'], x_key='epoch', file_name='loss.png'))
trainer.extend(extensions.PlotReport(['main/accuracy', 'validation/main/accuracy'], x_key='epoch', file_name='accuracy.png'))
trainer.extend(extensions.dump_graph('main/loss'))
trainer.run()

epoch       main/loss   main/accuracy  validation/main/loss  validation/main/accuracy  elapsed_time
1           2.31081     0.0991833      2.31142               0.0977                    10.0181       
2           2.30428     0.107733       2.30804               0.0949                    21.696        
